In [5]:
from dotenv import load_dotenv
load_dotenv()  # Loads from local .env by default

import os
groq_key = os.getenv("GROQ_API_KEY")

from groq import Groq
groq_client = Groq(api_key=groq_key)

print("Groq client initialized ✅")



Groq client initialized ✅


In [6]:
# TODO: Import traceable
from groq import Groq
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "groq"
MODEL_NAME = "llama-3.1-8b-instant" 
APP_VERSION = 1.1  # bumped version
RAG_SYSTEM_PROMPT = """You are a teaching-focused assistant.
Always explain clearly with short examples when possible.
If unsure, admit it politely and suggest where to learn more.
Use max three sentences.
"""

groq_client = Groq()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

# TODO: Set up tracing for each function
def retrieve_documents(question: str):
    return retriever.invoke(question)   # NOTE: This is a LangChain vector db retriever, so this .invoke() call will be traced automatically


def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_groq(messages)


def call_groq(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return groq_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )


def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [7]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

To trace with the `@traceable` decorator using LangSmith, you need to decorate a function with `@traceable`. 

For example:

```python
from langsmith import traceable

@traceable
async def my_function():
    # your code here
    pass
```

Additionally, make sure to set the `LANGSMITH_TRACING` environment variable to `'true'` to enable tracing.


In [9]:
from langsmith import traceable

@traceable(
    metadata={"vectordb": "faiss", "environment": "dev", "app_version": APP_VERSION},
    tags=["rag", "retrieval"]
)
def retrieve_documents(question: str):
    # TODO: replace this with your retriever logic
    return retriever.get_relevant_documents(question)


@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_groq(messages)


@traceable(
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER, "temperature": 0.0},
    tags=["llm", "completion"]
)
def call_groq(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return groq_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )


@traceable(tags=["rag", "pipeline"])
def langsmith_rag(question: str, **kwargs):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    answer = response.choices[0].message.content
    print(f"[DEBUG] Question: {question}\n[DEBUG] Answer: {answer}")  # Debug logging
    return answer


In [10]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

C:\Users\Lokesh Pareek\AppData\Local\Temp\ipykernel_21820\863029767.py:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return retriever.get_relevant_documents(question)


[DEBUG] Question: How do I add Metadata to a Run with @traceable?
[DEBUG] Answer: To add metadata to a Run with `@traceable()`, you can pass a `metadata` dictionary as an argument to the decorator or the `hello_llm` function. Here's an example:

```python
@traceable(
    run_type="llm",
    metadata={"ls_provider": "my_provider", "ls_model_name": "my_model", "other_key": "other_value"}
)
def hello_llm(prompt: str):
    return {
        "choices": [
            {"text": "Hello, " + prompt}
        ],
        "usage_metadata": {
            "input_tokens": 4,
            "output_tokens": 5,
            "total_tokens": 9,
        },
    }

hello_llm("polly the parrot\n")
```

In this example, `metadata={"ls_provider": "my_provider", "ls_model_name": "my_model", "other_key": "other_value"}` is passed to the `@traceable()` decorator, which will associate the metadata with the Run.
To add metadata to a Run with `@traceable()`, you can pass a `metadata` dictionary as an argument to the decora

In [11]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(
    question,
    langsmith_extra={"metadata": {"runtime_metadata": f"run_v{APP_VERSION}"}}
)


[DEBUG] Question: How do I add metadata at runtime?
[DEBUG] Answer: To add metadata at runtime, you can pass it as a parameter to the `invoke` method. For example:

```python
chain.invoke({"input": "What is the meaning of life?"}, {"metadata": {"invoke-key": "invoke-value"}})
```

This way, you can dynamically set the metadata for the invocation. Alternatively, you can update the metadata on the parent run tree as shown in the code snippet:

```python
rt = ls.get_current_run_tree()
rt.metadata["some-conditional-key"] = "some-val"
```

Note that the `rt` variable is obtained from the `ls.get_current_run_tree()` method, which gives you access to the current run tree. You can then update the metadata dictionary associated with it.
